In [190]:
# load API key into .env
from dotenv import load_dotenv
_ = load_dotenv() 
#print(LAMINI_API_KEY)

8e0c1d6b1457c58bae67816ba74db2c7595c8cdaed2ddffab535c3722f9acd81


In [192]:
import pandas as pd
import json
import logging
import sqlite3
import lamini
import pandas as pd
#import util
#import python_util
#from util.get_schema import get_schema
#from util.make_llama_3_prompt import make_llama_3_prompt
#from util.setup_logging import setup_logging

logger = logging.getLogger(__name__)
engine = sqlite3.connect("./EHR_SEQSQL.db")
#setup_logging()
lamini.api_key = LAMINI_API_KEY

In [194]:
with open('data/EHR-SeqSQL-data.json','r') as file:
    data = json.load(file) 
print(data[0:20]) # first 20 data from entry for visualization 
for key,val in data[0].items():
    print(key,type(val)) # key, and their data types


[{'id': '908b5b18d58416e49d3ea2ba', 'seed_question': 'what are the methods of consumption of send 500mg vial?', 'value': {'drug_name': 'send 500mg vial'}, 'question': ['what are the methods of consumption of send 500mg vial?'], 'question_template': ['What is the intake method of send 500mg vial?'], 'sql': ["select distinct prescriptions.route from prescriptions where prescriptions.drug = 'send 500mg vial'"], 'seqsql': ["select distinct prescriptions.route from prescriptions where prescriptions.drug = 'send 500mg vial'"], 'department': "['nursing']", 'importance': 'medium', 'compositional_split': 'train', 'random_split': 'train'}, {'id': '007cc09dea68f1a13eb5eee0', 'seed_question': 'what are the ways to consume caspofungin?', 'value': {'drug_name': 'caspofungin'}, 'question': ['what are the ways to consume caspofungin?'], 'question_template': ['What is the intake method of caspofungin?'], 'sql': ["select distinct prescriptions.route from prescriptions where prescriptions.drug = 'caspofu

In [196]:
llm = lamini.Lamini(model_name="meta-llama/Meta-Llama-3-8B-Instruct")

In [198]:
def get_schema():
    return """\
0|id|TEXT
1|seed_question|TEXT
2|value|TEXT
3|question|TEXT
4|question_template|TEXT
5|sql|TEXT
6|seqsql|TEXT
7|department|TEXT
8|importance|TEXT
9|compositional_split|TEXT
10|random_split|TEXT
"""
# define our schema from the data provided

In [200]:
# Meta Llama 3 Instruct uses a prompt template, with special tags used to indicate the user query and system prompt. 
# You can find the documentation on this [model card](https://llama.meta.com/docs/model-cards-and-prompt-formats/meta-llama-3/#meta-llama-3-instruct).
def make_llama_3_prompt(user, system=""):
    system_prompt = ""
    if system != "":
        system_prompt = (
            f"<|start_header_id|>system<|end_header_id|>\n\n{system}<|eot_id|>"
        )
    return f"<|begin_of_text|>{system_prompt}<|start_header_id|>user<|end_header_id|>\n\n{user}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

In [202]:
user1 = """Is there a specific way to take insulin human regular?"""
user2 = """Can you explain the way to consume sterile water?"""


In [204]:
system = f"""You are a clinical data analyst with extensive experience in generating SQL queries for drug prescription data. Consider the prescriptions table with the following schema:
{get_schema()}
Write a sqlite query to answer the following question. Follow instructions exactly"""

In [206]:
print(system)

You are a clinical data analyst with extensive experience in generating SQL queries for drug prescription data. Consider the prescriptions table with the following schema:
0|id|TEXT
1|seed_question|TEXT
2|value|TEXT
3|question|TEXT
4|question_template|TEXT
5|sql|TEXT
6|seqsql|TEXT
7|department|TEXT
8|importance|TEXT
9|compositional_split|TEXT
10|random_split|TEXT

Write a sqlite query to answer the following question. Follow instructions exactly


In [208]:
prompt = make_llama_3_prompt(user1, system)

In [210]:
print(llm.generate(prompt, max_new_tokens=200))

To answer this question, we can use the following SQL query:

```sql
SELECT *
FROM prescriptions
WHERE question = 'Is there a specific way to take insulin human regular?';
```

This query will return all the rows from the `prescriptions` table where the `question` column matches the given question.


To answer this question, we can use the following SQL query:

```sql
SELECT *
FROM prescriptions
WHERE question = 'Is there a specific way to take insulin human regular?';
```

This query will return all the rows from the `prescriptions` table where the `question` column matches the given question.


In [214]:
prompt = make_llama_3_prompt(user2, system)

In [216]:
print(llm.generate(prompt, max_new_tokens=200))

Based on the provided schema, I'll assume that the question "Can you explain the way to consume sterile water?" is stored in the "seed_question" column. To answer this question, we need to find the corresponding SQL query in the "sql" column that is associated with this seed question.

Here's the SQLite query to achieve this:
```sql
SELECT sql
FROM prescriptions
WHERE seed_question = 'Can you explain the way to consume sterile water?';
```
This query will return the SQL query that is associated with the given seed question.
